In [15]:
import numpy as np
import os
import pathlib
import cv2
import matplotlib.pyplot as plt
import random
import math
from sys import exit
import h5py
import pygame
from pygame.locals import *
import pickle
import math
import pathlib
import torch
from torch import nn
from torch.nn import functional as F
import torchvision.transforms

In [16]:
t_steps = 10
s_path = 'D:/test_dump/test_new/00002/'
traj_path = 'D:/traj_test/00002/'
eps = os.listdir(traj_path)
iters = ['1','2','3','4','5','6','7','8','9','10']

In [17]:
def cosine_beta_schedule(timesteps, s=0.008, dtype=torch.float32):
    """
    cosine schedule
    as proposed in https://openreview.net/forum?id=-NEXDKk8gZ
    """
    steps = timesteps + 1
    x = np.linspace(0, steps, steps)
    alphas_cumprod = np.cos(((x / steps) + s) / (1 + s) * np.pi * 0.5) ** 2
    alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
    betas_clipped = np.clip(betas, a_min=0, a_max=0.999)
    return betas_clipped

In [18]:
def q_sample(x_start, t, self_alpha, self_minus_alpha,i, noise):
    """
    Diffuse the data for a given number of diffusion steps.
    In other words, sample from q(x_t | x_0).
    :param x_start: the initial data batch.
    :param t: the number of diffusion steps (minus 1). Here, 0 means one step.
    :param noise: if specified, the split-out normal noise.
    :return: A noisy version of x_start.
    """
    if noise is None:
        noise = torch.randn_like(x_start)
    assert noise.shape == x_start.shape
    return (
        x_start
        + _extract_into_tensor(self_minus_alpha, t, x_start.shape)
        * noise*.075
    )

In [19]:
def _extract_into_tensor(arr, timesteps, broadcast_shape):
    """
    Extract values from a 1-D numpy array for a batch of indices.
    :param arr: the 1-D numpy array.
    :param timesteps: a tensor of indices into the array to extract.
    :param broadcast_shape: a larger shape of K dimensions with the batch
                            dimension equal to the length of timesteps.
    :return: a tensor of shape [batch_size, 1, ...] where the shape has K dims.
    """
    res = torch.from_numpy(arr).to(device=timesteps.device)[timesteps].float()
    while len(res.shape) < len(broadcast_shape):
        res = res[..., None]
    return res.expand(broadcast_shape)

In [22]:
save_path = s_path
for i, ep in enumerate(eps[:1]):
    path_c = save_path+ep
    pathlib.Path(path_c).mkdir(parents = True, exist_ok = True)
    for j in iters[0]:
        path_cc = path_c+'/'+j
        pathlib.Path(path_cc).mkdir(parents = True, exist_ok = True)
        
        i_path = traj_path+ep+'/'+j
        #print(i_path)
        with h5py.File(i_path+'/Coordinates.h5', 'r') as hf:
            keys = list(hf.keys())
            bl_c = np.array(hf['black_coords'])
            gr_c = np.array(hf['green_coords'])
            gr_r = np.array(hf['green_radius'])
            red_c = np.array(hf['red_coords'])
            red_r = np.array(hf['red_radius'])
            tar_c = np.array(hf['target_coords'])
            
        x_green = torch.from_numpy(gr_c/256)
        radius = gr_r[0]/256
        timesteps = t_steps
        betas = cosine_beta_schedule(timesteps)
        alphas = 1. - betas
        alphas_cumprod = np.cumprod(alphas, axis=0)
        self_alpha = np.sqrt(alphas_cumprod)
        self_minus_alpha = np.sqrt(1. - alphas_cumprod)

        x_diff = []
        x_t = x_green
        for t in list(range(timesteps))[::]:
            t_batch = torch.tensor([t] * 1, device=x_t.device)
            noise = torch.randn_like(x_t)
            noise = torch.clamp(noise,-1,1)
            x_t = q_sample(x_t, t_batch, self_alpha, self_minus_alpha,t, noise)
            x_diff.append(x_t.numpy().clip(radius,1.0-radius))
            
        ln = 256
        im_path = path_cc+'/'


        for j in range(len(x_diff)):
            
            path_ccc = im_path+str(j)
            pathlib.Path(path_ccc).mkdir(parents = True, exist_ok = True)
            image1 = path_ccc+"/sub.jpg"
            
            pygame.init()
            window1 = pygame.display.set_mode((ln, ln))
            window1.fill((255, 255, 255))
            
            for i in range(gr_c.shape[0]):

                pygame.draw.circle(window1, (0,255,0), x_diff[j][i]*256, gr_r[0], 0)
                
                
            pygame.display.update()
            pygame.image.save(window1, image1)
            pygame.quit()    
                
                
            image2 = im_path+str(j)+"/rest.jpg"

            pygame.init()    
            window2 = pygame.display.set_mode((ln, ln))    
            window2.fill((255, 255, 255))            
            
            for k in range(len(x_diff)):
                
                if k != j:
                    
                    for i in range(gr_c.shape[0]):
                        
                        pygame.draw.circle(window2, (0,255,0), x_diff[k][i]*256, gr_r[0], 0)
                    
                
                
            pygame.display.update()
            pygame.image.save(window2, image2)
            pygame.quit()      
                

            image3 = im_path+str(j)+"/env.jpg"

            pygame.init()    
            window3 = pygame.display.set_mode((ln, ln))    
            window3.fill((255, 255, 255))     
                
                
            pygame.draw.line(window3, (0,0,0),bl_c[0], bl_c[1],7)
            pygame.draw.line(window3, (0,0,255),tar_c[0], tar_c[1],10)                    
                
            pygame.display.update()
            pygame.image.save(window3, image3)
            pygame.quit()        

In [ ]:
save_path = s_path
for i, ep in enumerate(eps[:1]):
    path_c = save_path+ep
    pathlib.Path(path_c).mkdir(parents = True, exist_ok = True)
    for j in iters[0]:
        path_cc = path_c+'/'+j
        pathlib.Path(path_cc).mkdir(parents = True, exist_ok = True)
        
        i_path = traj_path+ep+'/'+j
        #print(i_path)
        with h5py.File(i_path+'/Coordinates.h5', 'r') as hf:
            keys = list(hf.keys())
            bl_c = np.array(hf['black_coords'])
            gr_c = np.array(hf['green_coords'])
            gr_r = np.array(hf['green_radius'])
            red_c = np.array(hf['red_coords'])
            red_r = np.array(hf['red_radius'])
            tar_c = np.array(hf['target_coords'])
            
        x_green = torch.from_numpy(gr_c/256)
        radius = gr_r[0]/256
        timesteps = t_steps
        betas = cosine_beta_schedule(timesteps)
        alphas = 1. - betas
        alphas_cumprod = np.cumprod(alphas, axis=0)
        self_alpha = np.sqrt(alphas_cumprod)
        self_minus_alpha = np.sqrt(1. - alphas_cumprod)

        x_diff = []
        x_t = x_green
        for t in list(range(timesteps))[::]:
            t_batch = torch.tensor([t] * 1, device=x_t.device)
            noise = torch.randn_like(x_t)
            noise = torch.clamp(noise,-1,1)
            x_t = q_sample(x_t, t_batch, self_alpha, self_minus_alpha,t, noise)
            x_diff.append(x_t.numpy().clip(radius,1.0-radius))
            
        ln = 256
        im_path = path_cc+'/'


        for j in range(len(x_diff)):
            
            path_ccc = im_path+str(j+1)
            pathlib.Path(path_ccc).mkdir(parents = True, exist_ok = True)
            for i in range(len(x_diff[j])):
                path_cccc = path_ccc+'/'+str(i+1)
                pathlib.Path(path_cccc).mkdir(parents = True, exist_ok = True)
                image1 = path_cccc+"/main.jpg"      
                
                pygame.init()
                window1 = pygame.display.set_mode((ln, ln))
                window1.fill((255, 255, 255))
                pygame.draw.circle(window1, (0,255,0), x_diff[j][i]*256, gr_r[0], 0)
                pygame.display.update()
                pygame.image.save(window1, image1)
                pygame.quit()    
                
                
                image2 = path_cccc+"/rest.jpg"

                pygame.init()    
                window2 = pygame.display.set_mode((ln, ln))    
                window2.fill((255, 255, 255))            

                for k in range(len(x_diff[j])):

                    if k != i:

                        pygame.draw.circle(window2, (0,255,0), x_diff[j][k]*256, gr_r[0], 0)
                    
                
                
                pygame.display.update()
                pygame.image.save(window2, image2)
                pygame.quit()      


                image3 = path_cccc+"/env.jpg"

                pygame.init()    
                window3 = pygame.display.set_mode((ln, ln))    
                window3.fill((255, 255, 255))     


                pygame.draw.line(window3, (0,0,0),bl_c[0], bl_c[1],7)
                pygame.draw.line(window3, (0,0,255),tar_c[0], tar_c[1],10)                    

                pygame.display.update()
                pygame.image.save(window3, image3)
                pygame.quit()        

In [ ]:
save_path = s_path
for i, ep in enumerate(eps[:1]):
    path_c = save_path+ep
    pathlib.Path(path_c).mkdir(parents = True, exist_ok = True)
    for j in iters[0]:
        path_cc = path_c+'/'+j
        pathlib.Path(path_cc).mkdir(parents = True, exist_ok = True)
        
        i_path = traj_path+ep+'/'+j
        #print(i_path)
        with h5py.File(i_path+'/Coordinates.h5', 'r') as hf:
            keys = list(hf.keys())
            bl_c = np.array(hf['black_coords'])
            gr_c = np.array(hf['green_coords'])
            gr_r = np.array(hf['green_radius'])
            red_c = np.array(hf['red_coords'])
            red_r = np.array(hf['red_radius'])
            tar_c = np.array(hf['target_coords'])
            
        x_green = torch.from_numpy(gr_c/256)
        radius = gr_r[0]/256
        timesteps = t_steps
        betas = cosine_beta_schedule(timesteps)
        alphas = 1. - betas
        alphas_cumprod = np.cumprod(alphas, axis=0)
        self_alpha = np.sqrt(alphas_cumprod)
        self_minus_alpha = np.sqrt(1. - alphas_cumprod)

        x_diff = []
        x_t = x_green
        for t in list(range(timesteps))[::]:
            t_batch = torch.tensor([t] * 1, device=x_t.device)
            noise = torch.randn_like(x_t)
            noise = torch.clamp(noise,-1,1)
            x_t = q_sample(x_t, t_batch, self_alpha, self_minus_alpha,t, noise)
            x_diff.append(x_t.numpy().clip(radius,1.0-radius))
            
        ln = 256
        im_path = path_cc+'/'
        
        count = 0
        
        for j in range(len(x_diff)):
            
            for i in range(len(x_diff[j])):
                
                count = count+1
                path_ball = im_path+str(count)
                
                image1 = path_ball+"/ball.jpg"      
                
                pygame.init()
                window1 = pygame.display.set_mode((ln, ln))
                window1.fill((255, 255, 255))
                pygame.draw.circle(window1, (0,255,0), x_diff[j][i]*256, gr_r[0], 0)
                pygame.display.update()
                pygame.image.save(window1, image1)
                pygame.quit()    
                
                
                image2 = path_ball+"/rest.jpg"

                pygame.init()    
                window2 = pygame.display.set_mode((ln, ln))    
                window2.fill((255, 255, 255))            

                for k in range(len(x_diff[j])):

                    if k != i:

                        pygame.draw.circle(window2, (0,255,0), x_diff[j][k]*256, gr_r[0], 0)
                    
                
                
                pygame.display.update()
                pygame.image.save(window2, image2)
                pygame.quit()      


                image3 = path_cccc+"/env.jpg"

                pygame.init()    
                window3 = pygame.display.set_mode((ln, ln))    
                window3.fill((255, 255, 255))     


                pygame.draw.line(window3, (0,0,0),bl_c[0], bl_c[1],7)
                pygame.draw.line(window3, (0,0,255),tar_c[0], tar_c[1],10)                    

                pygame.display.update()
                pygame.image.save(window3, image3)
                pygame.quit()        